# Using the Datashader Legends

Legends decode visualizations. This notebook reviews use of the `datashader.bokeh_ext.create_ramp_legend` and `datashader.bokeh_ext.create_categorical_legend`. Both are helper functions which return Bokeh plots to support understanding of how values map to colors within a 'datashaded' image.

In the following examples, we create legend to summarized a static image generated by datashader.  To see additional examples of using dynamic legends which update on zooming and panning, refer to https://github.com/bokeh/datashader/blob/master/examples/dashboard/dashboard.py#L283-L297

In [ ]:
from bokeh.io import output_notebook, show
from bokeh.plotting import Figure

import pandas as pd
import datashader as ds
import datashader.transfer_functions as tf

from datashader.colors import Hot
from datashader.bokeh_ext import create_ramp_legend, create_categorical_legend

output_notebook()

### Load NYC Taxi Data

In [ ]:
df = pd.read_csv('data/nyc_taxi.csv', usecols=['dropoff_x', 'dropoff_y', 'tpep_dropoff_datetime'])

small amount of data prep to classify dropoff times into morning, afternoon, and evening

In [ ]:
df['hour'] = pd.to_datetime(df['tpep_dropoff_datetime']).dt.hour
df['time'] = pd.cut(df['hour'], bins=[0, 12, 17, 24], labels=['morning', 'afternoon', 'evening']).astype('category')

### Simple datashaded image displayed using Bokeh.ImageRGBA

In [ ]:
def create_base_plot():
    
    # taxi data is in meters
    xmin = -8240227.037
    ymin = 4974203.152
    xmax = -8231283.905
    ymax = 4979238.441

    cvs = ds.Canvas(plot_width=900,
                    plot_height=600,
                    x_range=(xmin, xmax),
                    y_range=(ymin, ymax))

    agg = cvs.points(df, 'dropoff_x', 'dropoff_y')
    img = tf.shade(agg, cmap=Hot, how='log')
    fig = Figure(x_range=(xmin, xmax),
                 y_range=(ymin, ymax),
                 plot_width=900,
                 plot_height=600,
                 tools='')
    
    fig.background_fill_color = 'black'
    fig.toolbar_location = None
    fig.axis.visible = False
    fig.grid.grid_line_alpha = 0
    fig.min_border_left = 0
    fig.min_border_right = 0
    fig.min_border_top = 0
    fig.min_border_bottom = 0

    fig.image_rgba(image=[img.data],
                   x=[xmin],
                   y=[ymin],
                   dw=[xmax-xmin],
                   dh=[ymax-ymin])
    return fig, (xmin, ymin, xmax, ymax), agg

fig, extent, datashader_agg = create_base_plot()
show(fig)


Above is a base datashaded image of NYC Taxi data as a starting place to explore creating legends.  We are visualizing the number of dropoffs at a given location.

Since we are using a `log` transfer function to color the dropoff locations, let's now create a corresponding legend

In [ ]:
legend_fig = create_ramp_legend(datashader_agg,
                                Hot,
                                how='log',
                                width=900)
show(legend_fig)

The `create_ramp_legend` `how` argument accepts any of the how methods used in `tf.shade`.

### Creating a legend for categorical image

In [ ]:
# taxi data is in meters
xmin = -8240227.037
ymin = 4974203.152
xmax = -8231283.905
ymax = 4979238.441

cvs = ds.Canvas(plot_width=900,
                plot_height=600,
                x_range=(xmin, xmax),
                y_range=(ymin, ymax))

In [ ]:
colors = {'morning':'lime', 'afternoon':'magenta', 'evening':'cyan'}
cvs = ds.Canvas(plot_width=800, plot_height=500, x_range=(xmin, xmax), y_range=(ymin, ymax))
agg = cvs.points(df, 'dropoff_x', 'dropoff_y', ds.count_cat('time'))
img = tf.shade(agg, color_key=colors, how='eq_hist')

In [ ]:
fig = Figure(x_range=(xmin, xmax),
             y_range=(ymin, ymax),
             plot_width=900,
             plot_height=600,
             tools='')

fig.background_fill_color = 'black'
fig.toolbar_location = None
fig.axis.visible = False
fig.grid.grid_line_alpha = 0
fig.min_border_left = 0
fig.min_border_right = 0
fig.min_border_top = 0
fig.min_border_bottom = 0

fig.image_rgba(image=[img.data],
               x=[xmin],
               y=[ymin],
               dw=[xmax-xmin],
               dh=[ymax-ymin])

fig.background_fill_color = 'black'
fig.toolbar_location = None
fig.axis.visible = False
fig.grid.grid_line_alpha = 0
fig.min_border_left = 0
fig.min_border_right = 0
fig.min_border_top = 0
fig.min_border_bottom = 0

fig.image_rgba(image=[img.data],
               x=[xmin],
               y=[ymin],
               dw=[xmax-xmin],
               dh=[ymax-ymin])

show(fig)

Now that we have a categorical image, let's create a categorical legend

In [ ]:
cat_legend = create_categorical_legend(colors)

# you can also provide aliases for legend keys:
aliases = {}
aliases['morning'] = 'Morning (00:00-12:00)'
aliases['afternoon'] = 'Afternoon (12:00-19:00)'
aliases['evening'] = 'Evening (19:00-00:00)'
cat_legend_with_aliases = create_categorical_legend(colors, aliases=aliases)
show(cat_legend)
show(cat_legend_with_aliases)

While `create_ramp_legend` and `create_categorical_legend` are great tools for creating simple 'lookups' for the color values in your plot, they certainly don't cover all the types of legends you may need.  To create more complex legends, like a multivariant legend or a radial legend, check out the Bokeh.